## ImageSynthesizer Example

This notebook demonstrates how to use the `ImageSynthesizer` for generating image-based test cases:
- Uses image generation models (Gemini Imagen, DALL-E, etc.) to create images
- Creates test cases for evaluating generated images with `ImageJudge`
- Stores images directly in the database for persistence
- Provides rich metadata including generation prompts and expected outputs

The `ImageSynthesizer` is designed for:
- **Image Generation Testing**: Generate images from text prompts and evaluate quality
- **Visual Content Validation**: Create test sets for image generation models
- **Automated Visual QA**: Build pipelines for testing image generation capabilities

Key Configuration Parameters:
- `prompt`: Text description for image generation
- `model`: Image generation model (e.g., "gemini/imagen-3.0-generate-002")
- `expected_output`: Description of what generated images should contain
- `category`, `topic`, `behavior`: Classification for generated tests
- `image_size`: Dimensions of generated images (default: "1024x1024")
- `batch_size`: Maximum tests per generation batch (default: 5)


### Setup: Configure API Credentials

**Important**: Image generation models have different availability:
- **Vertex AI Imagen** (`vertex_ai/imagegeneration@006`): Requires Google Cloud project with Vertex AI enabled
- **Gemini Imagen** (`gemini/imagen-3.0-generate-002`): Currently in **private preview** - most accounts don't have access

This notebook uses **Vertex AI** which is the most reliable option.


In [ ]:
# Set up your API credentials and configuration
import os

from rhesis.sdk.synthesizers import ImageSynthesizer
from rhesis.sdk.metrics import ImageJudge

# Configure your API credentials

# For Vertex AI Imagen (recommended - more reliable):
# os.environ["VERTEX_AI_PROJECT"] = "your-gcp-project-id"
# os.environ["VERTEX_AI_LOCATION"] = "us-central1"
# GOOGLE_APPLICATION_CREDENTIALS should point to your service account JSON

# For Gemini API (required for text_model):
# os.environ["GEMINI_API_KEY"] = "your-gemini-api-key"

# For Rhesis backend (if pushing test sets):
# os.environ["RHESIS_API_KEY"] = "your-rhesis-api-key"

print("✓ SDK configured successfully")
print("Ready to generate image test cases with ImageSynthesizer!")


## Example 1: Basic ImageSynthesizer Usage

Generate simple image tests with default settings.


In [ ]:
# Create a basic image synthesizer
# Using Vertex AI Imagen for image generation (more reliable than Gemini API)
# text_model is used for generating test set properties (name, description)
synthesizer = ImageSynthesizer(
    prompt="Images of crashed cars for insurance damage documentation",
    model="vertex_ai/imagegeneration@006",   # Vertex AI Imagen for image generation
    text_model="gemini/gemini-2.0-flash",    # Gemini for text generation
    expected_output="Image should contain a crashed car, with damage to the body and windows"
)

# Generate image tests
result = synthesizer.generate(num_tests=5)

print(f"Generated {len(result.tests)} image tests")
print(f"Test set type: {result.test_set_type}")
print(f"Test set metadata keys: {list(result.metadata.keys())}")


In [ ]:
# Inspect the structure of generated tests
first_test = result.tests[0]

print("Test Structure:")
print(f"- Test Type: {first_test.test_type}")
print(f"- Behavior: {first_test.behavior}")
print(f"- Category: {first_test.category}")
print(f"- Topic: {first_test.topic}")
print(f"- Has binary data: {first_test.test_binary is not None}")
if first_test.test_binary:
    print(f"- Binary size: {len(first_test.test_binary)} bytes")

print(f"\nMetadata keys: {list(first_test.metadata.keys())}")
print(f"- MIME type: {first_test.metadata.get('binary_mime_type')}")
print(f"- Generation prompt: {first_test.metadata.get('generation_prompt')[:80]}...")
print(f"- Expected output: {first_test.metadata.get('expected_output')}")
print(f"- Model used: {first_test.metadata.get('model')}")


### Display Generated Images

View the generated images directly in the notebook.


In [ ]:
from IPython.display import display, Image as IPImage

# Display generated images
for i, test in enumerate(result.tests):
    if test.test_binary:
        print(f"\n=== Image {i+1} ===")
        print(f"Prompt: {test.metadata.get('generation_prompt', 'N/A')}")
        
        # Display the image
        display(IPImage(data=test.test_binary))
        print(f"Size: {len(test.test_binary)} bytes")


## Example 2: Evaluating Generated Images with ImageJudge

Use the `ImageJudge` metric to evaluate the quality of generated images.


In [ ]:
# Create an ImageJudge for evaluation
judge = ImageJudge(
    model="gemini/gemini-2.0-flash",  # Vision-capable model for evaluation
    categories=["pass", "partial", "fail"],
    passing_categories=["pass"]
)

print("=== Evaluating Generated Images ===")
print(f"Judge model: {judge.model.model_name}")
print(f"Categories: {judge.config.categories}")
print(f"Passing categories: {judge.config.passing_categories}")
print()

# Evaluate each generated image
for i, test in enumerate(result.tests):
    if test.test_binary:
        # Evaluate the image directly with bytes (no conversion needed)
        evaluation = judge.evaluate(
            input=test.metadata.get('generation_prompt', ''),
            output=test.test_binary,
            expected_output=test.metadata.get('expected_output', ''),
            output_mime_type=test.metadata.get('binary_mime_type', 'image/png')
        )
        
        # Determine if passed based on score being in passing categories
        passed = evaluation.score in judge.config.passing_categories
        
        print(f"Image {i+1}:")
        print(f"  Score: {evaluation.score}")
        print(f"  Passed: {passed}")
        print(f"  Reason: {evaluation.details.get('reason', 'N/A')[:150]}...")
        print()


## Example 4: Pushing Test Sets to Backend

Save the generated image tests to the Rhesis backend for persistent storage.

In [ ]:
# First, set up the test set properties
result.name = "Car Crash Image Tests"
result.description = "Generated image tests for crashed cars"
result.short_description = "Crashed cars"

print("Test Set Properties:")
print(f"  Name: {result.name}")
print(f"  Description: {result.description}")
print(f"  Test count: {len(result.tests)}")
print(f"  Test type: {result.test_set_type}")

# Uncomment to push to backend:
response = result.push()
print(f"\nPushed test set with ID: {result.id}")

## Example 5: Saving Generated Images to Disk

Export the generated images to local files for inspection.


In [ ]:
from pathlib import Path

# Create output directory
output_dir = Path("generated_images")
output_dir.mkdir(exist_ok=True)

print(f"Saving images to: {output_dir.absolute()}")

# Save each generated image
saved_count = 0
for i, test in enumerate(result.tests):
    if test.test_binary:
        # Determine file extension from MIME type
        mime_type = test.metadata.get('binary_mime_type', 'image/png')
        ext = mime_type.split('/')[-1]
        if ext == 'jpeg':
            ext = 'jpg'
        
        # Save the image
        filename = output_dir / f"image_{i+1}.{ext}"
        with open(filename, 'wb') as f:
            f.write(test.test_binary)
        
        print(f"Saved: {filename} ({len(test.test_binary)} bytes)")
        saved_count += 1

print(f"\n✓ Saved {saved_count} images to {output_dir}")
